# Python setup

## Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

In [ ]:
from buildingspy.io.outputfile import Reader

In [ ]:
from ipypublish import nb_setup

## Cosmetics


In [ ]:
pd = nb_setup.setup_pandas()
pd.set_option('precision', 3)

In [ ]:
# https://stackoverflow.com/a/39566040/11552622
rcparams = {
    'axes.titlesize':13,
    'axes.labelsize':9,
    'xtick.labelsize':8,
    'ytick.labelsize':8
}

In [ ]:
# IPyPublish customization of matplotlib.pyplot and pandas
from ipypublish import nb_setup
plt = nb_setup.setup_matplotlib(output=('pdf','svg'), rcparams=rcparams, usetex=True)

## Path configuration

In [ ]:
file_names = {
    # Results summary table
    'table': 'table.csv', 
    # Hemodynamic parameters
    'Xexact': 'Xexact.csv', 'Xpred':'Xpredicted.csv',
    # DNN responses
    'Ytest':'Ytest.txt', 'Ypred':'Ytestpred.txt',
    # Simulation files
    'simulation': 'Ursino1998Model_VAD2_{}_{}.mat'
}

In [ ]:
prefix = '/media/maousi/Data/tmp/'
suffix = '/results/'
results_folders = {
    # Boolean: artificial pulse
    False: {
        # Integer: RPM
        4000: 'simulations_2020_03_21',
        5000: 'simulation_LVAD_RPM5000_2020_04_21',
        6000: 'simulation_LVAD_RPM6000_2020_04_22',
        # Convention: put the simulations without LVAD here, i.e. RPM=0
        0:    'simulation_noLVAD_2020_04_15'
    },
    True: {
        # Integer: RPM
        4000: 'simulation_LVAD_RPM4000_Pulse_T30_N2000_2020_04_26',
        5000: 'simulation_LVAD_RPM5000_Pulse_T30_N2000_2020_04_26',
        6000: 'simulation_LVAD_RPM6000_Pulse_T30_N2000_2020_04_26'
        #4000: 'simulation_LVAD_RPM4000_Pulse_T20_N500_2020_05_31',
        #5000: 'simulation_LVAD_RPM5000_Pulse_T20_N500_2020_05_31',
        #6000: 'simulation_LVAD_RPM6000_Pulse_T20_N500_2020_05_31'
    }
}
for ap, dic in results_folders.items():
    for rpm, path in dic.items():
        dic[rpm] = prefix+path+suffix

---

# Latex generator

## Parameter range

In [ ]:
cols = [r'$E_{max,lv}$', r'$E_{max,lv,0}$',
                  r'$G_{E_{max,lv}}$', r'$k_{E,lv}$']
index = ['Lower bound', 'Upper bound']
data = np.array([
    [0.2, 2.95],
    [0.2, 2.392],
    [0.2, 0.475],
    [0.011, 0.014]
]).transpose()

In [ ]:
pd.DataFrame(data=data, index=index, columns=cols)

## DNN results

In [ ]:
# Define utility function to format results table
def format_table(table):
    table.index = [
        'Min exact', 'Max exact', 'Mean exact', 'SD exact', 'Min pred', 
        'Max pred', 'Mean pred', 'SD pred', 'Avg err mx', 
        'Avg rel err%', 'SD error', 'CI min', 'CI max',
    ]
    table.columns = [
        'HR','SAPM','SAPS','SAPD', 'PAPM','PAPS','PAPD','LVEF',
        'LVEDV', 'LVESV', 'CI', 'PCPW'
    ]
    # Multiply relative error by 100
    table.loc['Avg rel err%', :] = table.loc['Avg rel err%', :] * 100

In [ ]:
# Load the table, format it and return it
def get_results_table(artificial_pulse, RPM):
    """
    :param artificial_pulse: bool
    :param RPM: int
    """
    df = pd.read_csv(
        results_folders[artificial_pulse][RPM] + file_names['table'], 
        header=None
    )
    format_table(df)
    return df

In [ ]:
get_results_table(artificial_pulse=True, RPM=4000)

In [ ]:
def format_table_article(art_pulse, RPM):
    # TODO : check multicolumn param of to_latex !!!! and many other params
    df = get_results_table(artificial_pulse=art_pulse, RPM=RPM)
    return df.to_latex(caption=f'{art_pulse}, {RPM}', bold_rows=True, 
                       float_format='%.2f', escape=True)

In [ ]:
format_table_article(True,4000)

In [ ]:
from datetime import datetime

In [ ]:
configs = [(True, 4000), (True, 5000), (True, 6000), (False, 4000),
           (False, 5000), (False, 6000)]

data = '\n'.join([format_table_article(*config) for config in configs])

In [ ]:
header = '%\n% AUTOMATICALLY GENERATED - ' + datetime.now().strftime('%d-%m-%Y %H:%M:%S')
header += '\n% Notebook : N13 Article Utility\n%\n\n'
with open('data/dnn_hemodynamic_tables.tex', 'w') as f:
    f.write(header + data)

# Get a visual

## Normalization

In [ ]:
#import sys
#import scipy.io as sio

#sys.path.insert(0, '../Simulation_script/')

#%run ../Deep_learning/utils_deeplearning.py

#X = sio.loadmat('/media/maousi/Data/tmp/dnntest/X.mat')['X']

#X.shape

#plt.hist(X[0, 0])

#ncoeff, nfreq = X.shape[1], X.shape[2]

#newmins = np.full([ncoeff, nfreq], 0.0)
#newmaxs = np.full([ncoeff, nfreq], 1.0)

#%%capture cap_out
#Xnorm, _, _ = normalizeinputmatDL(X, newmins, newmaxs);

#plt.hist(Xnorm[0, 0])

# Figures



## SHF simulations

In [ ]:
sns.set(style='whitegrid')

In [ ]:
# Define: variables to load
variables = ['SystemicArteries.PC', 'AorticValve.Inlet.Q',
             'SystemicArteries.Inlet.Q', 'LVAD.RPM', 'LeftVentricle.Inlet.P']
# Define folder
folder_standalone_sim = '/media/maousi/Data/tmp/standalone_simulations/'

In [ ]:
# Data structure: nested dictionnaries
# level 1 = heart failure, level 2 = rpm
data_sim = {
    hf_level: {rpm: None for rpm in [4000, 5000, 6000]}
    for hf_level in ['MHF', 'SHF']
}

In [ ]:
def time_range(tmin, tmax, t, signal):
    ids = np.where(np.logical_and(t >= tmin, t <= tmax))
    return t[ids], signal[ids]

def read_simulations(path, file_format, data, variables, tmin, tmax):
    for hf_level, subdata in data.items():
        for rpm in subdata:
            file = path + file_format.format(hf_level, rpm)
            reader = Reader(file, 'dymola')
            subdata[rpm] = [
                time_range(tmin, tmax, *reader.values(v)) for v in variables
            ]

In [ ]:
read_simulations(
    path=folder_standalone_sim, file_format='Ursino1998Model_VAD2_{}_{}.mat',
    data=data_sim, variables=variables, tmin=26.0, tmax=30.0
)

In [ ]:
data_sim['SHF'][4000]

In [ ]:
# We have 2 heart failure levels * 3 rpm levels = 6 simulations
# The function handles data for a given heart failure level
def plot_simulations(data_hf):
    fig, ax = plt.subplots(len(data_hf), 1, figsize=(10, 10))

    for i, (rpm_c, data) in enumerate(data_hf.items()):
        ax1 = ax[i]
        ax2 = ax1.twinx()

        (t1, pressure), (t2, valve_flow), (t3, aortic_flow), (t4, rpm), (t5, lv_pressure) = data
        #(t1, pressure), (t2, valve_flow), (t3, arotic_flow), (t4, rpm) = data[0], data[1], data[2], data[3]
        ax1.plot(t4, rpm/100, 'k--', linewidth=1)
        ax1.plot(t1, pressure, 'b')
        ax1.plot(t1, lv_pressure, '--', linewidth=0.5)
        ax2.plot(t2, valve_flow, 'g--', linewidth=1)
        ax2.plot(t3, aortic_flow, 'r')
        #ax2.set_ylim([min(flow)-25, max(flow)+25])
        #ax1.set_ylim(-5, max(max(pressure), max(flow)))
        #ax2.plot(t2, flow)

        # Manage x axis
        if i == 2:
            ax1.set_xlabel('Time [s]')
        else:
            pass#ax1.set_xticks([])
        
        ax1.set_ylabel('Systemic pressure [mmHg]\nLV pressure [mmHg]\nPump speed [RPM/100]')
        ax2.set_ylabel('Aortic flow [ml/s]\nValve flow [ml/s]')
        #ax1.yaxis.label.set_color('b')
        # Align the grids
        #ax2.set_yticks(np.linspace(ax2.get_yticks()[0], ax2.get_yticks()[-1], len(ax1.get_yticks())))
        ax2.grid(False)
        ax2.set_ylim([-5, 1.5*max(aortic_flow)])
        #ax1.set_title('RPM mean = {}'.format(rpm_c))

    plt.subplots_adjust(hspace=.2)

In [ ]:
plot_simulations(data_sim['SHF'])
plt.savefig('figs/SHF_simulations.eps')

In [ ]:
def plot_simulations2(data_hf):
    fig, ax = plt.subplots(len(data_hf), 1, figsize=(10, 10))

    for i, (rpm_c, data) in enumerate(data_hf.items()):
        ax1 = ax[i]
        ax2 = ax1.twinx()

        (t1, pressure), (t2, valve_flow), (t3, aortic_flow), (t4, rpm), (t5, lv_pressure) = data
        #(t1, pressure), (t2, valve_flow), (t3, arotic_flow), (t4, rpm) = data[0], data[1], data[2], data[3]
        l1, = ax1.plot(t4, rpm/100, 'k--', linewidth=1)
        l2, = ax1.plot(t1, pressure, 'b')
        l3, = ax1.plot(t1, lv_pressure, '--', linewidth=0.5)
        l4, = ax2.plot(t2, valve_flow, 'g--', linewidth=1)
        l5, = ax2.plot(t3, aortic_flow, 'r')
        #ax2.set_ylim([min(flow)-25, max(flow)+25])
        #ax1.set_ylim(-5, max(max(pressure), max(flow)))
        #ax2.plot(t2, flow)

        # Manage x axis
        if i == 2:
            ax1.set_xlabel('Time [s]')
        else:
            pass#ax1.set_xticks([])
        
        ax1.set_ylabel('Systemic pressure [mmHg]\nLV pressure [mmHg]\nPump speed [RPM/100]')
        ax2.set_ylabel('Aortic flow [ml/s]\nValve flow [ml/s]')
        #ax1.yaxis.label.set_color('b')
        # Align the grids
        #ax2.set_yticks(np.linspace(ax2.get_yticks()[0], ax2.get_yticks()[-1], len(ax1.get_yticks())))
        ax2.grid(False)
        ax2.set_ylim([-5, 1.5*max(aortic_flow)])
        #ax1.set_title('RPM mean = {}'.format(rpm_c))

    plt.subplots_adjust(hspace=.2)
    ax2.legend(handles = [l1,l2,l3, l4, l5], labels=['LVAD speed', 'SAP', 'LV pressure', 'Aortic valve flow', 'Aortic flow'], 
               bbox_to_anchor=(0.5, -0.35), fancybox=False, shadow=False, ncol=5, loc='center')

In [ ]:
plot_simulations2(data_sim['SHF'])
plt.savefig('figs/SHF_simulations_enhanced.eps')

### Compare pressure 


### Compute metrics

In [ ]:
shf = data_sim['SHF']

In [ ]:
# get_sap returns (t, pressure)
get_sap = lambda datastruct, rpm: datastruct[rpm][0]

In [ ]:
# Mean SAP


## DNN accuarcy

### Effect of artificial pulse

In [ ]:
Ymse = pickle.load(open('data/dnn_mse.bin', 'rb'))

In [ ]:
Ymse

In [ ]:
Ymse.index

In [ ]:
YmseT = Ymse.transpose()
YmseT

In [ ]:
rpms = [4000, 5000, 6000]
columns = pd.Index(name='RPM', data=rpms)
artpulse_error = pd.DataFrame(columns=columns, index=YmseT.index)
for rpm in rpms:
    # Compute MSE relative increase of each param for each rpm
    percincrease = (YmseT[(rpm, True)] - YmseT[(rpm, False)])/YmseT[(rpm, False)] * 100
    artpulse_error.loc[:, rpm] = percincrease

In [ ]:
artpulse_error

In [ ]:
artpulse_error.apply(np.mean, axis=1)

### Effect of LVAD presence

In [ ]:
Ymse.drop((0, False)).apply(np.mean)

## HEmodynamic quantities

In [ ]:
X = pd.read_csv('data/hemodynamic_test_data.csv')

In [ ]:
X

In [ ]:
X.columns

In [ ]:
# Relative errors
E = X.loc[:, ['RPM', 'art_pulse'] + [col for col in X.columns if '_relerr' in col]]

In [ ]:
E

### Mean relative errors

In [ ]:
E

In [ ]:
E = E.groupby(['RPM', 'art_pulse']).mean().rename({
    col: col.split('_')[0] for col in E.columns[2:]
}, axis=1)

In [ ]:
E.index.rename(['RPM', 'Art. Pulse'], inplace=True)

In [ ]:
pd.set_option('precision', 2)

In [ ]:
E

### Mean

In [ ]:
Xmean = X.loc[:, [c for c in X.columns if '_relerr' in c]].mean()
Xmean

### Median

In [ ]:
Xmed = X.loc[:, [c for c in X.columns if '_relerr' in c]].median()
Xmed

In [ ]:
pd.DataFrame([Xmean, Xmed]).transpose().rename({0:'Mean', 1:'Median'}, axis=1)

### Describe

In [ ]:
X.loc[:, [c for c in X.columns if '_relerr' in c]].describe().transpose()

### Effect of artificial pulse

In [ ]:
AP_data_ = E.drop('CI', axis=1).drop((0, False))
AP_data_

In [ ]:
AP_data = pd.DataFrame(index=AP_data_.columns)
for rpm in [4000,5000,6000]:
    diff = AP_data_.loc[(rpm, True), :] - AP_data_.loc[(rpm, False), :]
    AP_data[rpm] = diff / AP_data_.loc[(rpm, True), :] * 100

In [ ]:
AP_data

In [ ]:
AP_data.mean()

### Effet of RPMs when artificial pulse is active

In [ ]:
RPM = X.loc[X.art_pulse == True, ['RPM'] + [c for c in X.columns if '_relerr' in c]].groupby('RPM').mean()
RPM

### Effect of RPM when artificial pulse is NOT active

In [ ]:
RPM = X.loc[X.art_pulse == False, ['RPM'] + [c for c in X.columns if '_relerr' in c]].groupby('RPM').mean()
RPM